In [1]:
import cv2 as cv
import numpy as np
import matplotlib.pyplot as plt
import copy

In [3]:
def load_templates(path):
    numbers = [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 24, 25, 27, 28, 30, 32, 35, 36, 40, 42, 45, 48, 49, 50, 54, 56, 60, 63, 64, 70, 72, 80, 81, 90]
    templates = []
    for number in numbers:
        template = cv.imread(f'{path}/{number}.jpg')
        template_grey = cv.cvtColor(template, cv.COLOR_BGR2GRAY)
        templates.append((number, template_grey))
    # print(f'Loaded {len(templates)} templates')
    # max_height = max([template[1].shape[1] for template in templates])
    # print(f'Max height: {max_height}')
    return templates

In [4]:
def get_initial_board_state():
    empty_board = [
    ['3x', '', '', '', '', '', '3x', '3x', '', '', '', '', '', '3x'],
    ['', '2x', '', '', ':', '', '', '', '', ':', '', '', '2x', ''],
    ['', '', '2x', '', '', '-', '', '', '-', '', '', '2x', '', ''],
    ['', '', '', '2x', '', '', '+', '*', '', '', '2x', '', '', ''],
    ['', ':', '', '', '2x', '', '*', '+', '', '2x', '', '', ':', ''],
    ['', '', '-', '', '', '', '', '', '', '', '', '-', '', ''],
    ['3x', '', '', '*', '+', '', 1, 2, '', '*', '+', '', '', '3x'],
    ['3x', '', '', '+', '*', '', 3, 4, '', '+', '*', '', '', '3x'],
    ['', '', '-', '', '', '', '', '', '', '', '', '-', '', ''],
    ['', ':', '', '', '2x', '', '+', '*', '', '2x', '', '', ':', ''],
    ['', '', '', '2x', '', '', '*', '+', '', '', '2x', '', '', ''],
    ['', '', '2x', '', '', '-', '', '', '-', '', '', '2x', '', ''],
    ['', '2x', '', '', ':', '', '', '', '', ':', '', '', '2x', ''],
    ['3x', '', '', '', '', '', '3x', '3x', '', '', '', '', '', '3x']] 
    return empty_board

In [5]:
board_size = 14
moves_per_game = 50
operators = ['+', '-', '*', ':']
multipliers = ['2x', '3x']
column_letter_mapping = {
    0: 'A',
    1: 'B',
    2: 'C',
    3: 'D',
    4: 'E',
    5: 'F',
    6: 'G',
    7: 'H',
    8: 'I',
    9: 'J',
    10: 'K',
    11: 'L',
    12: 'M',
    13: 'N'
}
templates = load_templates('templates')

In [6]:
def plot_image(image):
    plt.imshow(image, cmap='gray')
    plt.axis('off')
    plt.show()

def plot_images(fig, images):
    n_images = len(images)
    cols = 4
    rows = (n_images + cols - 1) // cols 

    for i, (image, title) in enumerate(images, 1):
        ax = fig.add_subplot(rows, cols, i)
        ax.imshow(image)
        ax.set_title(title)
        ax.axis('off')  

    plt.tight_layout()
    plt.show()

In [7]:
def extract_board(image):
    # Preprocess
    blue_channel = image[:, :, 0]
    median_blurred_img = cv.medianBlur(blue_channel, 3)
    gaussian_blurred_img = cv.GaussianBlur(median_blurred_img, (7, 7), 0) 
    # Segmentate
    edge_enhanced_img = cv.addWeighted(median_blurred_img, 1.5, gaussian_blurred_img, -0.5, 0)
    _, mask = cv.threshold(edge_enhanced_img, 80, 255, cv.THRESH_BINARY)
    # Refine
    kernel = np.ones((5, 5), np.uint8)
    mask_cleaned1 = cv.morphologyEx(mask, cv.MORPH_CLOSE, kernel)
    mask_cleaned2 = cv.morphologyEx(mask_cleaned1, cv.MORPH_OPEN, kernel)
    mask_cleaned3 = cv.erode(mask_cleaned2, kernel, iterations=4)
    # Extract
    contours, _ = cv.findContours(mask_cleaned3, cv.RETR_EXTERNAL, cv.CHAIN_APPROX_SIMPLE) 
    contours = sorted(contours, key=cv.contourArea, reverse=True)
    board_contour = contours[0]
    for contour in contours:
        x, y, w, h = cv.boundingRect(contour)
        aspect_ratio = w / h
        if 0.8 < aspect_ratio < 1.2: 
            board_contour = contour
            break
        
    x, y, w, h = cv.boundingRect(board_contour)
    cropped_board = image[y:y+h, x:x+w]
    resized_cropped_board = cv.resize(cropped_board, (1900, 1900))

    x_start, y_start = 250, 250  
    x_end, y_end = 1650, 1650  

    play_area = resized_cropped_board[y_start:y_end, x_start:x_end]
    resized_play_area = cv.resize(play_area, (1400, 1400))

    # Uncomment to see the intermediate results
    # img_with_all_contours = cv.drawContours(image.copy(), contours, -1, (255, 0, 0), 10)
    # img_with_board_contour = cv.drawContours(image.copy(), board_contour, -1, (255, 0, 0), 10) 
    # fig = plt.figure(figsize=(12, 12))
    # images = [
    #     (image, "Original Image"),
    #     (median_blurred_img, "Median Blur"),
    #     (gaussian_blurred_img, "Gaussian Blur"),
    #     (edge_enhanced_img, "Edge enchancement"),
    #     (mask, "Mask"),
    #     (mask_cleaned1, "Mask Cleaned Closing"),
    #     (mask_cleaned2, "Mask Cleaned Opening"),
    #     (mask_cleaned3, "Mask Cleaned Erosion"),
    #     (img_with_all_contours, "Img with all contours"),
    #     (img_with_board_contour, "Img with largest contour"),
    #     (resized_cropped_board, "Resized cropped board"),
    #     (resized_play_area, "Resized play area")
    # ]
    # plot_images(fig, images)
    return resized_play_area
   
    

In [8]:
def get_position_of_newly_added_piece(prev_board, curr_board):
    prev_board_gray = cv.cvtColor(prev_board, cv.COLOR_BGR2GRAY)
    curr_board_gray = cv.cvtColor(curr_board, cv.COLOR_BGR2GRAY)
    
    tile_size = 100  
    max_diff = 0
    # Uncomment to see the top 5 tiles detected by the algorithm with the highest difference between the previous and current board
    # images = []
    for i in range(board_size):
        for j in range(board_size):
            tile_x_start = i * tile_size
            tile_y_start = j * tile_size
            tile_x_end = (i + 1) * tile_size
            tile_y_end = (j + 1) * tile_size
            
            tile_from_prev_board = prev_board_gray[tile_y_start:tile_y_end, tile_x_start:tile_x_end]
            tile_from_curr_board = curr_board_gray[tile_y_start:tile_y_end, tile_x_start:tile_x_end]

            diff = cv.absdiff(tile_from_prev_board, tile_from_curr_board)
            diff_erosion = cv.erode(diff, np.ones((3,3), np.uint8), iterations=1)
            # images.append(diff_erosion)
            diff_mean = np.mean(diff_erosion)

            if diff_mean > max_diff:
                max_diff = diff_mean
                new_piece_position = (j,i)
                
    # images.sort(key=lambda x: np.mean(x), reverse=True)
    # for img in images[:5]:
    #     plot_image(img)
    return new_piece_position

In [9]:
def get_content_of_piece(board, position, templates):
    i, j = position
    board = cv.cvtColor(board, cv.COLOR_BGR2GRAY)
    piece_x_start = j * 100
    piece_x_end = piece_x_start + 100
    piece_y_start = i * 100
    piece_y_end = piece_y_start + 100
    piece = board[piece_y_start:piece_y_end, piece_x_start:piece_x_end]
    piece = cv.resize(piece, (150, 150))

    max_similariy = -1
    for number, template in templates:
        _, similarity, _, _ = cv.minMaxLoc(cv.matchTemplate(piece, template, cv.TM_CCOEFF_NORMED))
        if similarity > max_similariy:
            max_similariy = similarity
            content = number
    return content
    

In [10]:
def position_and_content(position, content):
    i, j = position
    return f'{i+1}{column_letter_mapping[j]} {content}'


In [11]:
def is_in_bounds(i, j):
    return 0 <= i < board_size and 0 <= j < board_size

def is_valid_equation(board_state, term1_pos, term2_pos, result):
    i_term1, j_term1 = term1_pos
    i_term2, j_term2 = term2_pos

    if not is_in_bounds(i_term1, j_term1) or not is_in_bounds(i_term2, j_term2):
        return False, ''

    term1 = board_state[i_term1][j_term1]
    term2 = board_state[i_term2][j_term2]
    if not type(term1) == int or not type(term2) == int:
        return False, ''

    for operator in operators:
        if operator == '+':
            if result == term1 + term2:
                return True, operator
        elif operator == '-':
            if result == term1 - term2 or result == term2 - term1:
                return True, operator
        elif operator == '*':
            if result == term1 * term2:
                return True, operator
        elif operator == ':':
            if (term2 != 0 and result == term1 // term2) or (term1 != 0 and result == term2 // term1):
                return True, operator
    return False, ''

def get_score_for_move(board_state, new_piece_position, new_piece_content):
    i, j = new_piece_position
    tile_before_adding_new_piece = board_state[i][j]
    score = 0

    prev_vertical_eq, op1 = is_valid_equation(board_state, (i-2, j), (i-1, j), new_piece_content)
    next_vertical_eq, op2 = is_valid_equation(board_state, (i+1, j), (i+2, j), new_piece_content)
    prev_horizontal_eq, op3 = is_valid_equation(board_state, (i, j-2), (i, j-1), new_piece_content)
    next_horizontal_eq, op4 = is_valid_equation(board_state, (i, j+1), (i, j+2), new_piece_content)
    
    check_equations = [(prev_vertical_eq, op1), (next_vertical_eq, op2), (prev_horizontal_eq, op3), (next_horizontal_eq, op4)]
    valid_equations = [eq for eq in check_equations if eq[0]]

    if tile_before_adding_new_piece in operators:
        valid_equations = [eq for eq in valid_equations if eq[1] == tile_before_adding_new_piece]

    number_of_valid_equations = sum([1 for eq in valid_equations])
    score = number_of_valid_equations * new_piece_content

    if tile_before_adding_new_piece in multipliers:
        multiplier = int(tile_before_adding_new_piece[0])
        score = score * multiplier
        
    return score

In [15]:
number_of_games = 4

def get_next_player(current_player):
    return '2' if current_player == '1' else '1'

def resolve_game(game):
    correct_positions, correct_content = 0, 0
    game_turns = open(f'antrenare/{game}_turns.txt', 'r').read().split("\n")
    actual_scores = open(f'antrenare/{game}_scores.txt', 'r').read().split("\n")
    rounds = len(actual_scores)
    result_scores = []
    current_player = game_turns[0].split(" ")[0][-1]
    prev_change_move = int(game_turns[0].split(" ")[1])
    scores = {
        '1': 0,
        '2': 0
    }
    change_player_at_moves = [int(turn.split(" ")[1]) for turn in game_turns[1:]]
    board_state = get_initial_board_state()
    prev_img, curr_img = None, None
    for move in range(1, moves_per_game+1):
        if move == 1:
            prev_img = cv.imread('imagini_auxiliare/01.jpg')
            curr_img = cv.imread(f'antrenare/{game}_0{move}.jpg')
            actual = open(f'antrenare/{game}_0{move}.txt', 'r').read()
        else:
            append_zero_to_prev = '0' if move <= 10 else ''
            append_zero_to_curr = '0' if move < 10 else ''
            prev_img = cv.imread(f'antrenare/{game}_{append_zero_to_prev}{move-1}.jpg')
            curr_img = cv.imread(f'antrenare/{game}_{append_zero_to_curr}{move}.jpg')
            actual = open(f'antrenare/{game}_{append_zero_to_curr}{move}.txt', 'r').read()

        prev_board = extract_board(prev_img)
        curr_board = extract_board(curr_img)
        
        position = get_position_of_newly_added_piece(prev_board, curr_board)
        content = get_content_of_piece(curr_board, position, templates)
        
        current_board_state = copy.deepcopy(board_state)
        score = get_score_for_move(current_board_state, position, content)
        
        if move in change_player_at_moves:
            result_scores.append(f'Player{current_player} {prev_change_move} {scores[current_player]}')
            prev_change_move = move
            current_player = get_next_player(current_player)
            scores[current_player] = 0
 
        scores[current_player] += score
        if move == moves_per_game:
            result_scores.append(f'Player{current_player} {prev_change_move} {scores[current_player]}')
            
        board_state[position[0]][position[1]] = content

        actual_position, actual_content = actual.split(" ")
        result_position, result_content = position_and_content(position, content).split(" ")
        if result_position != actual_position:
            print(f'Expected position: {actual_position}, got: {result_position} for img {game}_{move}')
        else:
            correct_positions += 1
        
        if result_content != actual_content:
            print(f'Expected number: {actual_content}, got: {result_content} for img {game}_{move}')
        else:
            correct_content += 1

    correct_scores = 0
    for i in range(len(result_scores)):
        if result_scores[i] == actual_scores[i]:
            correct_scores += 1
    print(result_scores)
    print(actual_scores)
    print('Task1')
    print("Correct positions:")
    print(f'Correct: {correct_positions}/{moves_per_game}')
    print(f'Accuracy: {correct_positions/moves_per_game * 100}%') 

    print('Task2')
    print("Correct content:")
    print(f'Correct: {correct_content}/{moves_per_game}')
    print(f'Accuracy: {correct_content/moves_per_game * 100}%') 
    
    print('Task3')
    print("Correct scores:")
    print(f'Correct: {correct_scores}/{rounds}')
    print(f'Accuracy: {correct_scores/rounds * 100}%')

for game in range(1, number_of_games+1):
    print(f'Game {game}')
    resolve_game(game)
    print('-------------------')
    

Game 1
['Player1 1 152', 'Player2 5 170', 'Player1 11 28', 'Player2 14 35', 'Player1 18 16', 'Player2 21 30', 'Player1 24 128', 'Player2 28 126', 'Player1 31 111', 'Player2 34 13', 'Player1 36 10', 'Player2 39 147', 'Player1 43 38', 'Player2 48 13']
['Player1 1 152', 'Player2 5 170', 'Player1 11 28', 'Player2 14 35', 'Player1 18 16', 'Player2 21 30', 'Player1 24 128', 'Player2 28 126', 'Player1 31 111', 'Player2 34 13', 'Player1 36 10', 'Player2 39 147', 'Player1 43 38', 'Player2 48 13']
Task1
Correct positions:
Correct: 50/50
Accuracy: 100.0%
Task2
Correct content:
Correct: 50/50
Accuracy: 100.0%
Task3
Correct scores:
Correct: 14/14
Accuracy: 100.0%
-------------------
Game 2
['Player2 1 309', 'Player1 7 94', 'Player2 13 84', 'Player1 16 12', 'Player2 19 52', 'Player1 23 220', 'Player2 29 62', 'Player1 33 55', 'Player2 39 46', 'Player1 41 18', 'Player2 44 112', 'Player1 47 16']
['Player2 1 309', 'Player1 7 94', 'Player2 13 84', 'Player1 16 12', 'Player2 19 52', 'Player1 23 220', 'Play